# 基本的な回路の同等性

In [1]:
from qiskit import *
from qiskit.circuit import Gate

量子コンピューターをプログラミングするとき、私たちの目標は常に、基本的なビルディングブロックから有用な量子回路を構築することです。ただし、必要とする基本的構成要素がすべて揃っていない場合もあります。このセクションでは、基本的なゲートを相互に変換する方法と、ゲートを使用して少し複雑な（ただしかなり基本的な）ゲートを構築する方法について説明します。

この章で説明する手法の多くは、1995年にバレンコと共著者による論文で最初に提案されました[1]。

### 制御$Z$ を CNOTから作成する

制御Zまたは`cz`ゲートは、よく使用されるまた別の2量子ビットゲートです。 CNOTは、その制御量子ビットが $|1\rangle$の状態にあるときはその標的量子ビットに$X$を適用しますが、同じように、制御$Z$は、同じ場合に𝑍を適用します。 Qiskitでは、制御$Z$を直接呼び出すことができます：


```python
# 制御Z
qc.cz(c,t)
```

ここで、cとtは制御量子ビットと標的量子ビットです。ただし、IBM Qデバイスでは、直接適用できる2量子ビットゲートはCNOTだけです。したがって、変換する方法が必要です。

このプロセスは非常に簡単です。アダマールが$|0\rangle$および$|1\rangle$の状態を$|+\rangle$および$|-\rangle$の状態に変換することはわかっています。 $|+\rangle$と$|-\rangle$の状態に対する𝑍ゲートの影響は、$|0\rangle$と$|1\rangle$の状態に対する$X$ の作用と同じであることもわかっています。このことから、または単純に行列を乗算することから、

$$
H X H = Z,\\\\
H Z H = X.
$$

同じトリックを使用して、CNOTを制御$Z$に変換できます。私たちがしなければならないのは、CNOTの前と後に、アダマールを標的量子ビットに作用させるだけです。これにより、その量子ビットに適用された$X$が$Z$に変換されます。

```python
# こちらも制御Z
qc.h(t)
qc.cx(c,t)
qc.h(t)
```

より一般的には、単一のCNOTを、正しい回転を単純に前後に配置することにより、ブロッホ球での角度$\pi$の回転の任意の制御に変換できます。たとえば、制御$Y$は：

```python
# 制御Y
qc.sdg(t)
qc.cx(c,t)
qc.s(t)
```

そして、制御$H$は：

```python
# 制御H
qc.ry(-pi/4,t)
qc.cx(c,t)
qc.ry(pi/4,t)
```

### 量子ビットのスワップ（交換）

量子コンピューターで情報を移動する必要がある場合があります。一部の量子ビットの実装では、これを物理的に移動することで実行できます。別のオプションでは、2つの量子ビット間で状態を移動することです。これはSWAPゲートによって行われます。


```python
# 量子ビットaとbの状態をスワップする
qc.swap(a,b)
```

上記のコマンドはこのゲートを直接呼び出しますが、標準のゲートセットを使用してそれを作成する方法を見てみましょう。このため、いくつかの例を検討する必要があります。

まず、量子ビットaが$|1\rangle$の状態で、量子ビットbが$|0\rangle$の状態である場合を見てみましょう。これには、次のゲートを適用します：

```python
# aの1をaからbに交換
qc.cx(a,b) # aからbへ1をコピーする
qc.cx(b,a) # bの1を使用してaの状態を0に回転
```

これは、量子ビットbを状態$|1\rangle$にし、量子ビットaを状態$|0\rangle$にする効果があります。この場合、少なくともSWAPを実行したことになります。

次に、この状態を取り、元の状態に戻します。ご想像のとおり、上記のプロセスの逆でこれを行うことができます：

```python
# qをbからaに入れ替える
qc.cx(b,a) # 1をbからaにコピーする
qc.cx(a,b) # aの1を使用してbの状態を0に回転
```

これらの2つのプロセスでは、一方の最初のゲートが他方の初期状態に影響を与えないことに注意してください。たとえば、bの$|1\rangle$をaに交換する場合、最初のゲートは`cx(b,a)`です。代わりに、これが最初にbに$|1\rangle$がなかった状態に適用された場合、効果はありません。

また、これらの2つのプロセスでは、一方の最終ゲートが他方の最終状態に影響を与えないことに注意してください。たとえば、$|1\rangle$をaからbにスワップするときに必要な最後の`cx(b,a)`は、$|1\rangle$がbにない状態には影響しません。

これらの観察結果を基に、一方のゲートからもう一方のゲートに無効なゲートを追加して、2つのプロセスを組み合わせることができます。例えば、

```python
qc.cx(b,a)
qc.cx(a,b)
qc.cx(b,a)
```

これは、aからbに$|1\rangle$を交換するプロセスと考えることができますが、最初は役に立たない `qc.cx(b,a)`があります。また、$|1\rangle$をbからaに交換するプロセスと考えることもできますが、最後に無用な`qc.cx(b,a)`があります。どちらにしても、結果は、双方向でスワップを実行できるプロセスです。

また、状態$|00\rangle$に対して正しく効果を発揮します。これは対称的であるため、状態を入れ替えても影響はありません。制御量子ビットが$|0\rangle$の場合、CNOTゲートは効果がないため、プロセスは何もしません。

状態$|11\rangle$も対称的であるため、スワップによる効果は取るに足りないものです。この場合、上記のプロセスの最初のCNOTゲートは2番目のゲートに影響を与えず、3番目のゲートは最初のゲートを元に戻します。したがって、全体の効果は確かに取るに足らないものです。

このようにして、SWAPゲートを標準ゲートセットである単一量子ビットの回転ゲートとCNOTゲートに分解する方法を見つけました。

```python
# 量子ビットaとbの状態のスワップ
qc.cx(b,a)
qc.cx(a,b)
qc.cx(b,a)
```

これは、$|00\rangle$、 $|01\rangle$、 $|10\rangle$、 $|11\rangle$、およびそれらのすべての重ね合わせに対して機能します。したがって、可能なすべての2量子ビット状態をスワップします。

CNOTゲートの順序を変更した場合も、同じ効果が得られます：

```python
# 量子ビットaとbの状態を入れ替えます
qc.cx(a,b)
qc.cx(b,a)
qc.cx(a,b)
```

これは、SWAPゲートを取得するための同様に有効な方法です。

ここで使用された導出は、z基底状態に非常に基づいていましたが、状態 $|+\rangle$ と $|-\rangle$でも同じように量子ビットのスワップができ、SWAPゲートを実装する方法は、完全に同じです。

### 持っているCNOTから必要なCNOTを作る

量子コンピューターのゲートは、そのシステムの物理学に従って動かされています。 IBM Qデバイスでは、その物理的性質から、CNOTは量子ビットのすべての可能なペアに直接適用できるわけではありません。 CNOTを適用できるペアについては、通常、特定の方向があります。 特定の量子ビットがコントロールとして機能し、もう1つがターゲットとして機能する必要があります。

#### CNOTの方向を変更する

上記の2番目の問題に対処しましょう。制御量子ビット$c$と標的量子ビット$t$のCNOTがある場合に、量子ビット$t$を制御として、量子ビット$c$を標的として機能させたい時、どのように作れば良いでしょうか？

この質問は、制御$Z$に対しては、非常に簡単に回答できます。 このゲートの場合、制御量子ビットと標的量子ビットがどのように配置されていてもかまいません。

```python
qc.cz(c,t)
```

は完全に以下の作用と同じです：

```python
qc.cz(t,c)
```

これは、どちらかを制御として、もう一方を標的として考えることができることを意味します。

これが真実である理由を確認するために、$Z$ゲートが何であるかを思い出してみましょう：

$$
Z= \begin{pmatrix} 1&0 \\\\ 0&-1 \end{pmatrix}.
$$

これは、$|1\rangle$の状態の時のみ、$-1$をかける作用と考えられます。

制御$Z$ゲートの場合、$Z$を標的量子ビットに適用するには、制御量子ビットの状態が $|1\rangle$である必要があります。 $Z$がこの状態の場合、標的が状態$|1\rangle$である場合にのみ効果があります。 したがって、制御$Z$ゲートは、2つの量子ビットの状態を$-1$で乗算するものと考えることができますが、それは状態が$|11\rangle$の場合のみです。

この新しい解釈は完全に対称的な方法で表現されており、「制御」および「標的」のラベルはこのゲートには必要ないことを示しています。

この性質は、CNOTの方向を逆にする方法を提供します。 最初に、前述の方法を使用して、CNOTを制御$Z$に変えることができます：アダマールを標的量子ビットの前後に配置します：

```python
# a cz
qc.h(t)
qc.cx(c,t)
qc.h(t)
```

次に、制御$Z$のアクションについて自由に選択できるため、$t$を制御として、$c$をターゲットとして考えることを選択できます。 次に、この制御$Z$を対応するCNOTに変換できます。標的量子ビット（量子ビット$c$）の前後にアダマールを配置するだけです。

```python
# 制御量子ビットtと標的量子ビットcを備えたcx
qc.h(c)
qc.h(t)
qc.cx(c,t)
qc.h(t)
qc.h(c)
```

これで、CNOTの向きが変わりました。 必要なのは、両方の量子ビットの前後のアダマールだけです。

このサブセクションの残りでは、ちょっとした数学を使った、CNOTの方向を変換する方法といくつかの異なる洞察について説明しています（前の章の'States for Many Qubits'の記事、およびこの章の'Fun with Matrices'の記事で紹介されています）。スキップして構いません。

CNOTゲートを別の表記方法を次に示します：

$$
{\rm CX}_{c,t} = |0\rangle \langle0| \otimes I + |1\rangle \langle1| \otimes X.
$$

$|1\rangle \langle1|$ は、第二項が、制御量子ビット$c$が重ね合わせの$|1\rangle$の部分のみに作用することを保証します。 その場合、標的量子ビット$t$への作用は$X$です。 最初の項は、制御量子ビットの重ね合わせの$|0\rangle$の部分についての作用ですが、この場合、標的量子ビットは影響を受けません。

それでは、少し数学をしてみましょう。 $X$ゲートは、状態$|+\rangle$および$|-\rangle$に対して固有値$\pm 1$を持っています。$I$ゲートの固有値は、$|+\rangle$と$|-\rangle$を含むすべての状態で$1$です。 したがって、それらをスペクトル形式で次のように書くことができます。

$$
X = |+\rangle \langle+| \, \, - \, \, |-\rangle \langle-|, \, \, \, \,  I = |+\rangle \langle+| \, \,  + \, \,  |-\rangle \langle-|
$$

これらを上記の式に代入すると、

$$
{\rm CX}_{c,t} = |0\rangle \langle0| \otimes |+\rangle \langle+| \, \,  + \, \, |0\rangle \langle0| \otimes |-\rangle \langle-| \, \,  + \, \, |1\rangle \langle1| \otimes |+\rangle \langle+| \, \,  - \, \, |1\rangle \langle1| \otimes |-\rangle \langle-|
$$

状態$|0\rangle$ と $|1\rangle$を使用して、$Z$ゲートをスペクトル形式で記述し、$I$のの別の（ただし完全に同等の）スペクトル形式を使用できます：

$$
Z = |0\rangle \langle0| ~-~ |1\rangle \langle1|, ~~~ I = |0\rangle \langle0| ~+~ |1\rangle \langle1|.
$$

これらを使用して、状態$|0\rangle$ と $|1\rangle$で表現された部分のCNOTを因数分解できます。

$$
{\rm CX}_{c,t} = I \otimes |+\rangle \langle+| \, \,  + \, \, Z \otimes |-\rangle \langle-|
$$

これにより、CNOTの影響を全く新しく解釈できます。 $Z \otimes |-\rangle \langle-| $ の項は、量子ビット$t$が状態$|-\rangle$にの重ね合わせの部分を示し、$Z$ゲートを量子ビット$c$に適用します。もう一方の項は、量子ビット$t$の状態が$|+\rangle$の場合ですが、量子ビット$c$には何も行いません。

この新しい解釈では、制御として機能するのは量子ビット$t$です。 アクションが実行されるかどうかを決定するのは$|+\rangle$ と $|-\rangle$の状態であり、そのアクションはゲート$Z$です。 これは、おなじみのCNOTとは非常に異なるゲートのように聞こえますが、それでもCNOTです。 これらは、その効果に関する2つの等しく正しい説明です。

この性質の多くの用途の1つに、CNOTを反転させる方法があります。 例えば、CNOTの前後で、アダマールを量子ビット$c$に適用することを考えてみましょう：

```python
h(c)
cx(c,t)
h(c)
```

これは$Z \otimes |-\rangle \langle-| $の項の$Z$を$X$に変換し、他の項は変換しません。 複合作用としては、量子ビット$t$が$|-\rangle$の状態のときに、量子ビット$c$に$X$を適用するゲートとなることです。 これは私たちが構築したいものの半分です。

プロセスを完了するために、量子ビット$t$の前後にアダマールを適用できます。 これにより、各項の$|+\rangle$ と $|-\rangle$の状態が$|0\rangle$ と $|1\rangle$に変換されます。 これで、量子ビット$t$が$|1\rangle$の状態のときに、量子ビット$c$に$X$を適用することになります。 これはまさに私たちが望んでいたことです：量子ビット$t$ が制御であり、$c$が標的となる、CNOTの逆向きです。

#### CNOT between distant qubits

Suppose we have a control qubit $c$ and a target qubit $t$, and we want to do a CNOT gate between them. If this gate is directly possible on a device, we can just do it. If it's only possible to do the CNOT in the wrong direction, we can use the method explained above. But what if qubits $c$ and $t$ are not connected at all?

If qubits $c$ and $t$ are on completely different devices in completely different labs in completely different countries, you may be out of luck. But consider the case where it is possible to do a CNOT between qubit $c$ and an additional qubit $a$, and it is also possible to do one between qubits $a$ and $t$. The new qubit can then be used to mediate the interaction between $c$ and $t$.

One way to do this is with the SWAP gate. We can simply SWAP $a$ and t, do the CNOT between $c$ and $a$, and then swap $a$ and $t$ back again. The end result is that we have effectively done a CNOT between $c$ and $t$. The drawback of this method is that it costs a lot of CNOT gates, with six needed to implement the two SWAPs.

Another method is to use the following sequence of gates.

```python
# a CNOT between qubits c and t, with no end effect on qubit a
qc.cx(a,t)
qc.cx(c,a)
qc.cx(a,t)
qc.cx(c,a)
```

To see how this works, first consider the case where qubit $c$ is in state $|0\rangle$. The effect of the `cx(c,a)` gates in this case are trivial. This leaves only the two `cx(a,t)` gates, which cancel each other out. The net effect is therefore that nothing happens.

If qubit $c$ is in state $|1\rangle$, things are not quite so simple. The effect of the `cx(c,a)` gates is to toggle the value of qubit $a$; it turns any $|0\rangle$ in the state of qubit $a$ into $|1\rangle$ and back again, and vice versa.

This toggle effect affects the action of the two `cx(a,t)` gates. It ensures that whenever one is controlled on a $|0\rangle$ and has trivial effect, the other is controlled on a $|1\rangle$ and applies an $X$ to qubit $t$. The end effect is that qubit $a$ is left unchanged, but qubit $t$ will always have had an $X$ applied to it.

Putting everything together, this means that an $X$ is applied to qubit $t$ only when qubit $c$ is in state $|1\rangle$. Qubit $a$ is left unaffected. We have therefore engineered a CNOT between qubits $c$ and $t$. Unlike when using SWAP gates, this required only four CNOT gates to implement.

It is similarly possible to engineer CNOT gates when there is a longer chain of qubits required to connect our desired control and target. The methods described above simply need to be scaled up.

### Controlled rotations

We have already seen how to build controlled $\pi$ rotations from a single CNOT gate. Now we'll look at how to build any controlled rotation.

First, let's consider arbitrary rotations around the y axis. Specifically, consider the following sequence of gates.

```python
qc.ry(theta/2,t)
qc.cx(c,t)
qc.ry(-theta/2,t)
qc.cx(c,t)
```

If the control qubit is in state $|0\rangle$, all we have here is a $R_y(\theta/2)$ immediately followed by its inverse, $R_y(-\theta/2)$. The end effect is trivial. If the control qubit is in state $|1\rangle$, however, the `ry(-theta/2)` is effectively preceded and followed by an X gate. This has the effect of flipping the direction of the y rotation and making a second $R_y(\theta/2)$. The net effect in this case is therefore to make a controlled version of the rotation $R_y(\theta)$. 

This method works because the x and y axis are orthogonal, which causes the x gates to flip the direction of the rotation. It therefore similarly works to make a controlled $R_z(\theta)$. A controlled $R_x(\theta)$ could similarly be made using CNOT gates.

We can also make a controlled version of any single-qubit rotation, $U$. For this we simply need to find three rotations A, B and C, and a phase $\alpha$ such that

$$
ABC = I, ~~~e^{i\alpha}AZBZC = U
$$

We then use controlled-Z gates to cause the first of these relations to happen whenever the control is in state $|0\rangle$, and the second to happen when the control is state $|1\rangle$. An $R_z(2\alpha)$ rotation is also used on the control to get the right phase, which will be important whenever there are superposition states.

```python
qc.append(a, [t])
qc.cz(c,t)
qc.append(b, [t])
qc.cz(c,t)
qc.append(c, [t])
qc.u1(alpha,c)
```

![A controlled version of a gate V](https://s3.us-south.cloud-object-storage.appdomain.cloud/strapi/4efe86a907a64a59a720b4dc54a98a88iden1.png)

Here `A`, `B` and `C` are gates that implement $A$ , $B$ and $C$, respectively, and must be defined as custom gates. For example, if we wanted $A$ to be $R_x(\pi/4)$, the custom would be defined as

```python
qc_a = QuantumCircuit(1, name='A')
qc_a.rx(np.pi/4,0)
A = qc_a.to_instruction()
```

### The Toffoli

The Toffoli gate is a three-qubit gate with two controls and one target. It performs an X on the target only if both controls are in the state $|1\rangle$. The final state of the target is then equal to either the AND or the NAND of the two controls, depending on whether the initial state of the target was $|0\rangle$ or $|1\rangle$. A Toffoli can also be thought of as a controlled-controlled-NOT, and is also called the CCX gate.

```python
# Toffoli with control qubits a and b and target t
qc.ccx(a,b,t)
```

To see how to build it from single- and two-qubit gates, it is helpful to first show how to build something even more general: an arbitrary controlled-controlled-U for any single-qubit rotation U. For this we need to define controlled versions of $V = \sqrt{U}$ and $V^\dagger$. In the code below, we assume that subroutines `cv` and `cvdg` have been defined for these, respectively. The controls are qubits $a$ and $b$, and the target is qubit $t$.

```python
qc.cv(b,t)
qc.cx(a,b)
qc.cvdg(b,t)
qc.cx(a,b)
qc.cv(a,t)
```

![A doubly controlled version of a gate V](https://s3.us-south.cloud-object-storage.appdomain.cloud/strapi/693974b222d24dba9111e02ae25e9151iden2.png)

By tracing through each value of the two control qubits, you can convince yourself that a U gate is applied to the target qubit if and only if both controls are 1. Using ideas we have already described, you could now implement each controlled-V gate to arrive at some circuit for the doubly-controlled-U gate. It turns out that the minimum number of CNOT gates required to implement the Toffoli gate is six [2].


![A Toffoli](https://s3.us-south.cloud-object-storage.appdomain.cloud/strapi/b3cbeb9b7d674d60a75bed351e4f2bcbiden3.png)


The Toffoli is not the unique way to implement an AND gate in quantum computing. We could also define other gates that have the same effect, but which also introduce relative phases. In these cases, we can implement the gate with fewer CNOTs.

For example, suppose we use both the controlled-Hadamard and controlled-$Z$ gates, which can both be implemented with a single CNOT. With these we can make the following circuit:

```python
qc.ch(a,t)
qc.cz(b,t)
qc.ch(a,t)
```

For the state $|00\rangle$ on the two controls, this does nothing to the target. For $|11\rangle$, the target experiences a $Z$ gate that is both preceded and followed by an H. The net effect is an $X$ on the target. For the states $|01\rangle$ and $|10\rangle$, the target experiences either just the two Hadamards \(which cancel each other out\) or just the $Z$ \(which only induces a relative phase\). This therefore also reproduces the effect of an AND, because the value of the target is only changed for the $|11\rangle$ state on the controls -- but it does it with the equivalent of just three CNOT gates.

### Arbitrary rotations from H and T

The qubits in current devices are subject to noise, which basically consists of gates that are done by mistake. Simple things like temperature, stray magnetic fields or activity on neighboring qubits can make things happen that we didn't intend.

For large applications of quantum computers, it will be necessary to encode our qubits in a way that protects them from this noise. This is done by making gates much harder to do by mistake, or to implement in a manner that is slightly wrong.

This is unfortunate for the single-qubit rotations $R_x(\theta)$, $R_y(\theta)$ and $R_z(\theta)$. It is impossible to implent an angle $\theta$ with perfect accuracy, such that you are sure that you are not accidentally implementing something like $\theta + 0.0000001$. There will always be a limit to the accuracy we can achieve, and it will always be larger than is tolerable when we account for the build-up of imperfections over large circuits. We will therefore not be able to implement these rotations directly in fault-tolerant quantum computers, but will instead need to build them in a much more deliberate manner.

Fault-tolerant schemes typically perform these rotations using multiple applications of just two gates: $H$ and $T$.

The T gate is expressed in Qiskit as

```python
qc.t(0) # T gate on qubit 0
```

It is a rotation around the z axis by $\theta = \pi/4$, and so is expressed mathematically as $R_z(\pi/4) = e^{i\pi/8~Z}$.

In the following we assume that the $H$ and $T$ gates are effectively perfect. This can be engineered by suitable methods for error correction and fault-tolerance.

Using the Hadamard and the methods discussed in the last chapter, we can use the T gate to create a similar rotation around the x axis.

```python
qc.h(0)
qc.t(0)
qc.h(0)
```

Now let's put the two together. Let's make the gate $R_z(\pi/4)~R_x(\pi/4)$.

```python
qc.h(0)
qc.t(0)
qc.h(0)
qc.t(0)
```

Since this is a single-qubit gate, we can think of it as a rotation around the Bloch sphere. That means that it is a rotation around some axis by some angle. We don't need to think about the axis too much here, but it clearly won't be simply x, y or z. More important is the angle.

The crucial property of the angle for this rotation is that it is irrational. You can prove this yourself with a bunch of math, but you can also see the irrationality in action by applying the gate. Repeating it $n$ times results in a rotation around the same axis by a different angle. Due to the irrationality, the angles that result from different repetitions will never be the same.

We can use this to our advantage. Each angle will be somewhere between $0$ and $2\pi$. Let's split this interval up into $n$ slices of width $2\pi/n$. For each repetition, the resulting angle will fall in one of these slices. If we look at the angles for the first $n+1$ repetitions, it must be true that at least one slice contains two of these angles. Let's use $n_1$ to denote the number of repetitions required for the first, and $n_2$ for the second.

With this, we can prove something about the angle for $n_2-n_1$ repetitions. This is effectively the same as doing $n_2$ repetitions, followed by the inverse of $n_1$ repetitions. Since the angles for these are not equal \(because of the irrationality\) but also differ by no greater than $2\pi/n$ \(because they correspond to the same slice\), the angle for $n_2-n_1$ repetitions satisfies

$$
\theta_{n_2-n_1} \neq 0, ~~~~-\frac{2\pi}{n} \leq \theta_{n_2-n_1} \leq \frac{2\pi}{n} .
$$

We therefore have the ability to do rotations around small angles. We can use this to rotate around angles that are as small as we like, just by increasing the number of times we repeat this gate.

By using many small-angle rotations, we can also rotate by any angle we like. This won't always be exact, but it is guaranteed to be accurate up to $2\pi/n$, which can be made as small as we like. We  now have power over the inaccuracies in our rotations.

So far, we only have the power to do these arbitrary rotations around one axis. For a second axis, we simply do the $R_z(\pi/4)$ and $R_x(\pi/4)$ rotations in the opposite order.

```python
qc.h(0)
qc.t(0)
qc.h(0)
qc.t(0)
```

The axis that corresponds to this rotation is not the same as that for the gate considered previously. We therefore now have arbitrary rotation around two axes, which can be used to generate any arbitrary rotation around the Bloch sphere. We are back to being able to do everything, though it costs quite a lot of $T$ gates.

It is because of this kind of application that $T$ gates are so prominent in quantum computation. In fact, the complexity of algorithms for fault-tolerant quantum computers is often quoted in terms of how many $T$ gates they'll need. This motivates the quest to achieve things with as few $T$ gates as possible. Note that the discussion above was simply intended to prove that $T$ gates can be used in this way, and does not represent the most efficient method we know.

### References

[1] [Barenco, *et al.* 1995](https://journals.aps.org/pra/abstract/10.1103/PhysRevA.52.3457?cm_mc_uid=43781767191014577577895&cm_mc_sid_50200000=1460741020)

[2] [Shende and Markov, 2009](http://dl.acm.org/citation.cfm?id=2011799)

In [2]:
import qiskit
qiskit.__qiskit_version__

{'qiskit-terra': '0.12.0',
 'qiskit-aer': '0.4.0',
 'qiskit-ignis': '0.2.0',
 'qiskit-ibmq-provider': '0.4.6',
 'qiskit-aqua': '0.6.4',
 'qiskit': '0.15.0'}